In [4]:
import jax.numpy as jnp
from jax import random, grad, jit
from jax.scipy.linalg import inv, svd, eigh, det

from jax.lax import scan
import jax
from scipy.linalg import solve_discrete_are
from tqdm import tqdm
import matplotlib.pyplot as plt
from jax.tree_util import Partial

from jax.numpy import linalg as jnpl
from tqdm.auto import tqdm


key = random.PRNGKey(3)

# System dimensions
n = 40  # System dimension
num_steps = 1000 # number of steps
J0 = 0 # burn in period
N = 10 # Monte Carlo samples

def create_stable_matrix(n, key):
    # Generate a symmetric random matrix
    A = random.normal(key, (n, n))
    A = (A + A.T) / 2
    
    # Ensure the matrix has a spectral radius < 1 for stability
    eigenvalues, eigenvectors = eigh(A)
    scaled_eigenvalues = eigenvalues / (jnp.abs(eigenvalues).max() + 0.1)  # Scale eigenvalues to ensure stability
    A_stable = eigenvectors @ jnp.diag(scaled_eigenvalues) @ eigenvectors.T
    
    return A_stable

# Model parameters
C0 = jnp.eye(n) * 1.0   # Initial state covariance matrix (P)
q = random.normal(key, (n, n))/5
Q = q@q.T + jnp.eye(n)*0.1  # Process noise covariance matrix (Sigma in Julia code)
H = jnp.eye(n)[::2] #partial observation
H = jnp.eye(n)
m0 = jnp.ones((n,))
R = jnp.eye(H.shape[0])  # R now becomes 20x20 for partial H 20*40
inv_R = inv(R)
M = create_stable_matrix(n,key)    # State transition matrix (A)
observation_interval = 1

# State initialization
vd0 = m0 + random.multivariate_normal(key, jnp.zeros(n), C0)


key, _ = random.split(key)


def state_transition_function(x):
    return jnp.dot(M, x)

def jacobian_function(x):
    return M

jac_func = Partial(jacobian_function)
A_step = Partial(state_transition_function)

In [5]:
H.shape

(40, 40)

In [6]:
from jax_vi import KL_gaussian, log_likelihood, KL_sum
from jax_filters import apply_filtering_fixed_linear, kalman_filter_process, filter_step_linear
from jax_models import visualize_observations, Lorenz96, generate_true_states, generate_gc_localization_matrix

In [7]:
observations, true_states = generate_true_states(key, num_steps, n, vd0, H, Q, R, A_step, observation_interval)

In [9]:
from functools import partial


@partial(jit, static_argnums=(3,10))
def var_cost(K, m0, C0, n, M, H, Q, R, y, key, N, num_steps, J0):
    states, covariances = apply_filtering_fixed_linear(m0, C0, y, K, n, M, H, Q, R)
    key, *subkeys = random.split(key, num=N+1)
    kl_sum = KL_sum(states, covariances, n, A_step, Q, key, N)

    def inner_map(subkey):
        return log_likelihood(random.multivariate_normal(subkey, states, covariances), y, H, R, num_steps, J0)  # Sometimes the covariances are negative definite. Fix
    cost = kl_sum - jnp.nanmean(jax.lax.map(inner_map, jnp.vstack(subkeys)))
    
    return cost


In [10]:
from functools import partial


@partial(jit, static_argnums=(3))
def var_cost_single_step(K, m0, C0, n, Q, H, R, y_curr, key, J, J0):
    (m_update, C_update), _  = filter_step_linear((m0, C0), y_curr, K, n, M, H, Q, R)
    
    log_likelihood_val = log_likelihood(m_update[jnp.newaxis, :], y_curr[jnp.newaxis, :], H, R, J=1, J0=J0)
    # Calculate the KL divergence between the predicted and updated state distributions
    m_pred = state_transition_function(m0)
    C_pred = M @ C0 @ M.T + Q
    kl_divergence = KL_gaussian(n, m_update, C_update, m_pred, C_pred)
    
    # Combine the KL divergence and the negative log-likelihood to form the cost
    cost = kl_divergence - log_likelihood_val
    return cost


In [11]:
P = solve_discrete_are(M.T, H.T, Q, R)
# Compute steady-state Kalman gain K
K_steady = P @ H.T @ jnp.linalg.inv(H @ P @ H.T + R)
# print("Steady-state K:")
#print(K_steady)



In [12]:
m, C, K = kalman_filter_process(A_step, jac_func, m0, C0, observations, H, Q, R)

(1000, 40, 40)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from jax.numpy import linalg as jnpl
from tqdm.auto import tqdm


true_div1 = []
prediction_errors1 = [] 
norms1 = []
Ks1 = []

live = True

# Define the gradient of the cost function
var_cost_single_grad = grad(var_cost_single_step, argnums = 0)

# Initial guess for K and optimization parameters
K_opt = jnp.eye(n) * 0.1
alpha = 1e-5

for i in tqdm(range(num_steps)):
    key, _ = random.split(key)
    y_curr = observations[i] 
    # Update the gradient and Kalman gain
    for j in range(100):
        grad_K = var_cost_single_grad(K_opt, m0, C0, n, Q, H, R, y_curr, key, num_steps, J0)
        K_opt -= alpha * grad_K
    Ks1.append(K_opt)
    norms1.append(jnp.linalg.norm(K_opt - K_steady)) 
    (m_update, C_update), _ = filter_step_linear((m0, C0), y_curr, K_opt, n, M, H, Q, R)
    prediction_error = np.square(m_update - true_states[i]).mean()  # Assuming true_states[i] is available
    prediction_errors1.append(prediction_error)
    true_div1.append(KL_gaussian(n, m_update, C_update, m[i], C[i]))
    # Prepare for the next step
    m0, C0 = m_update, C_update

num_iters = num_steps

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
last_200_errors_online = jnp.array(prediction_errors1[-200:])
print("Mean error for last 200 timesteps", jnp.mean(last_200_errors_online)

In [ ]:
# if H.shape[0] == 40:
#     K_opt = jnp.eye(n) * 0.4
# else: #partial observation
#     K_opt = jnp.zeros((40, 20))
#     for i in range(0, K_opt.shape[1]):
#         K_opt = K_opt.at[i*2, i].set(1)
#     K_opt = K_opt + random.normal(key, K_opt.shape) * 0.3


# true_div = []
# prediction_errors = [] 
# norms = []
# last_200_errors = []    # Errors for the last 200 timesteps
# Ks = []
# live = False


# t = 0  
# n_iters = 100
# var_cost_grad = grad(var_cost, argnums=0)
# # Initial guess for K and optimization parameters
# key, _ = random.split(key)
# alpha = 1e-5

# for i in tqdm(range(n_iters)):
#     key, _ = random.split(key)
#     # Update the gradient and Kalman gain
#     grad_K = var_cost_grad(K_opt, m0, C0, n, M, H, Q, R, observations, key, N, num_steps, J0)
#     K_opt -= alpha * grad_K
#     Ks.append(K_opt)
#     # Apply filtering with the newly optimized K to generate state predictions
#     predicted_states, covariances = apply_filtering_fixed_linear(m0, C0, observations, K_opt, n, M, H, Q, R)
#     prediction_error = jnp.mean(jnp.mean((predicted_states - true_states)**2, axis=1))#jnpl.norm(predicted_states - true_states) ** 2 / len(true_states)
#     prediction_errors.append(prediction_error)
#     last_200_prediction_error = jnp.mean((predicted_states[-200:] - true_states[-200:])**2)
#     last_200_errors.append(last_200_prediction_error)
#     norms.append(jnpl.norm(K_opt - K_steady))
#     total_kl_divergence = 0
#     for t in range(num_steps):  
#         kl_div_t = KL_gaussian(n, predicted_states[t], covariances[t],  m[t], C[t])
#         total_kl_divergence += kl_div_t
    
#     true_div.append(total_kl_divergence / num_steps)

In [ ]:
print("Mean error for last 200 timesteps (at convergence)", jnp.mean(jnp.array(last_200_errors[-10:])))

In [ ]:
import matplotlib.pyplot as plt
import os
fig, (ax1, ax3) = plt.subplots(figsize=(10, 4), ncols=2)
# Optimization Error
color = 'tab:red'
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Gain error ($\|K_\mathrm{opt} - K_\mathrm{steady}\|_F$)', color=color)
line1, = ax1.plot(range(1, n_iters+1), norms, label='Gain error', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Instantiate a second y-axis for Prediction Error and True Divergence
ax2 = ax1.twinx()
color_pred = 'tab:green'
ax2.set_ylabel('KL divergence to true filter', color=color_pred)
#line1, = ax2.plot(prediction_errors, label='Prediction Error (MSE)', color=color_pred, linestyle='--')
line2, = ax2.plot(range(1, n_iters+1), true_div, label='KL divergence to true filter', color=color_pred, linestyle='-.')
ax2.tick_params(axis='y', labelcolor=color_pred)

# Title and combined legend
#plt.title('Gain errors and KL divergence over iterations')

# Creating a combined legend for all lines
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes)

ax3.plot(range(1, n_iters+1), prediction_errors)
ax3.set_xlabel("Iteration")
ax3.set_ylabel("Prediction error (MSE)")

plt.tight_layout()
#plt.show()

subfolder_name = 'linear_results'
os.makedirs(subfolder_name, exist_ok=True)

file_base_name = "linear_gain"
if H.shape[0] == 20:
    file_base_name += "_partial"
if live:
    file_base_name += "_live"
file_name = file_base_name + ".pdf"

file_path = os.path.join(subfolder_name, file_name)
plt.savefig(file_path)


In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(10, 4), ncols=2)

ax1.pcolormesh(K_steady, cmap='RdBu', vmax=0.55, vmin=-0.55)
ax1.set_title('$K_\mathrm{steady}$')
p2 = ax2.pcolormesh(K_opt, cmap='RdBu', vmax=0.55, vmin=-0.55)
ax2.set_title('$K_\mathrm{opt}$')
cb_ax = fig.add_axes([.93,.124,.02,.754])
fig.colorbar(p2,orientation='vertical',cax=cb_ax)

subfolder_name = 'linear_results'
os.makedirs(subfolder_name, exist_ok=True)

file_base_name = "linear_gain_matrices"
if H.shape[0] == 20:
    file_base_name += "_partial"
if live:
    file_base_name += "_live"
file_name = file_base_name + ".pdf"

file_path = os.path.join(subfolder_name, file_name)
plt.savefig(file_path)
